In [ ]:
import requests
import time
import pandas as pd
from config import email, key
import matplotlib.pyplot as plt
import json

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
times_url = "&starttime=1990-01-01"
magnitude_url ="&minmagnitude=5.5"
geometry_url = "&minlatitude=25&maxlatitude=50&minlongitude=-125&maxlongitude=-65"

search = requests.get(base_url + times_url + magnitude_url + geometry_url)
response = search.json()

#print(json.dumps(response, indent=4, sort_keys=True))

response

In [ ]:
earthquakes_list = []
i=0

for record in response['features']:
    identifier = response['features'][i]['id']
    mag = response['features'][i]['properties']['mag']
    url = response['features'][i]['properties']['url']
    epochtime = response['features'][i]['properties']['time']
    four_weeks_before = epochtime - 2419200000
    four_weeks_after = epochtime +2419200000
    event_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
    month_before= time.strftime('%Y%m%d',  time.gmtime(four_weeks_before/1000))
    month_after = time.strftime('%Y%m%d',  time.gmtime(four_weeks_after/1000))
    converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
    timezone=response['features'][i]['properties']['tz']
    place = response['features'][i]['properties']['place']
    significance=response['features'][i]['properties']['sig']
    lon = response['features'][i]['geometry']['coordinates'][0]
    lat = response['features'][i]['geometry']['coordinates'][1]
    depth = response['features'][i]['geometry']['coordinates'][2]
    record_dict = {"Identifier":identifier, "Location":place, "Magnitude":mag, "Event Date":event_date,
                   "Month Before":month_before, "Month After":month_after, "Time":converted_time,"Time Zone":timezone, 
                   "Significance":significance, "Lat":lat, "Lon":lon, "Depth":depth, "Epoch Time":epochtime}
    earthquakes_list.append(record_dict)
    
    i += 1
    
earthquakes_df = pd.DataFrame(earthquakes_list)

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Mexico")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("MX")].reset_index()

earthquakes_df = earthquakes_df.fillna(value="")

earthquakes_abbrev =earthquakes_df[['Location', 'Event Date', 'Magnitude']]

pd.set_option('display.max_rows', None)

earthquakes_df.head()

In [ ]:
earthquakes_abbrev

# Select an EARTHQUAKE from list to Analyze (by index number):

In [ ]:
earthquake = int(input("Select earthquake by index:  "))

In [ ]:
# base_url finds air quality readings by Box

base_url = "https://aqs.epa.gov/data/api/dailyData/byBox?"

#parameters for byBox for CRITERIA Pollutants 
#42401="Sulfur Dioxide" 
#42602="Nitrogen Dioxide"
#42201="Ozone"
#42101 = "Carbon Monoxide"

begin = earthquakes_df.iloc[earthquake, 5]
end = earthquakes_df.iloc[earthquake, 6]
minlat=earthquakes_df.iloc[earthquake,10]-1
maxlat=earthquakes_df.iloc[earthquake,10]+1
minlon=earthquakes_df.iloc[earthquake,11]-1
maxlon=earthquakes_df.iloc[earthquake,11]+1
earthquake_location=earthquakes_df.iloc[earthquake,2]
event_date =earthquakes_df.iloc[earthquake, 4]

param_url = "param=42401,42602,44201,42101,81102,85129,88101,14129"
date_range_url = f"&bdate={begin}&edate={end}"
geometry_url = f"&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}"

url = (base_url + param_url + "&email=" + email + "&key=" + key + date_range_url + geometry_url)

In [ ]:
# Perform the search, jsonify the results, and store the data as "response"

print(color.BOLD + color.RED + f"Searching for data for Earthquake from location: {earthquake_location}.  Date of event: {event_date}." + color.END)

search = requests.get(url)
response = search.json()
response

In [ ]:
# iterate through the json data and establish a dataframe of the relevant fields

record_list = []
i=0

for record in response['Data']:
    site = response['Data'][i]['local_site_name']
    station = response['Data'][i]['site_number']
    parameter = response['Data'][i]['parameter']
    date = response['Data'][i]['date_local']
    #time = response['Data'][i]['time_local']
    #measurement = response['Data'][i]['sample_measurement']
    measurement = response['Data'][i]['arithmetic_mean']
    measurement_unit = response['Data'][i]['units_of_measure']
    event_type=response['Data'][i]["event_type"]
    lon = response['Data'][i]['longitude']
    lat = response['Data'][i]['latitude']
    record_dict = {"Parameter":parameter,"Date":date,"Station":station,"Site":site, "Lat":lat,"Lon":lon,
                   "Measurement":measurement, "Unit":measurement_unit,"Event Type":event_type}
    record_list.append(record_dict)

    i += 1
    
record_df = pd.DataFrame(record_list)

# also create dataframe grouped by Parameter and Collection Site

recordgroupby_df = record_df[['Parameter', 'Date', 'Measurement']]

recordgroupby_df = recordgroupby_df.groupby(['Parameter','Date'],as_index=False).mean()


recordbybox_df = record_df[['Site','Station','Date','Parameter','Lat','Lon','Measurement','Unit']]

recordbybox_df = recordbybox_df.sort_values(by=['Site','Date','Parameter'])

# save dataframes to a csv file. 

event_date =earthquakes_df.iloc[earthquake, 4]
location = earthquakes_df.iloc[earthquake, 2]
record_df.to_csv(f"DataFiles/{event_date}-{location}.csv")
recordbybox_df.to_csv(f"DataFiles/{event_date}-{location}_bySite.csv")
recordgroupby_df.to_csv(f"DataFiles/{event_date}-{location}_byPollutant.csv")
recordgroupby_df


In [ ]:
LeadTSPRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Lead (TSP) LC"]

LeadTSPPivot = pd.pivot_table(LeadTSPRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

LeadTSPPivot.to_csv(f"DataFiles/{event_date}-{location}_Sulfur.csv")
LeadTSPPivot

In [ ]:
LeadTSPRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Lead (TSP) LC"]

LeadTSPPivot = pd.pivot_table(LeadTSPRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

LeadTSPPivot.to_csv(f"DataFiles/{event_date}-{location}_Sulfur.csv")
LeadTSPPivot

In [ ]:
LeadPMRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Lead PM10 LC FRM/FEM"]

LeadPMPivot = pd.pivot_table(LeadPMRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

LeadPMPivot.to_csv(f"DataFiles/{event_date}-{location}_Sulfur.csv")
LeadPMPivot

In [ ]:
SulfurDioxideRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Sulfur dioxide"]

SulfurPivot = pd.pivot_table(SulfurDioxideRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

SulfurPivot.to_csv(f"DataFiles/{event_date}-{location}_Sulfur.csv")
SulfurPivot

In [ ]:
NitrogenDioxideRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Nitrogen dioxide (NO2)"]

NitrogenPivot = pd.pivot_table(NitrogenDioxideRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

NitrogenPivot.to_csv(f"DataFiles/{event_date}-{location}_NitrogenDioxide.csv")
NitrogenPivot

In [ ]:
OzoneRecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Ozone"]

OzonePivot = pd.pivot_table(OzoneRecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

OzonePivot.to_csv(f"DataFiles/{event_date}-{location}_Ozone.csv")

OzonePivot

In [ ]:
CORecordDates = recordbybox_df[recordbybox_df["Parameter"]=="Carbon monoxide"]

COPivot = pd.pivot_table(CORecordDates, values="Measurement", index = ["Station", "Site"], 
                             columns = ["Date"])

COPivot.to_csv(f"DataFiles/{event_date}-{location}_CarbonMonoxide.csv")

COPivot

In [ ]:
#Extract NO2 data from the groupby

Nitrogen_Dioxide = recordgroupby_df['Parameter']=="Nitrogen dioxide (NO2)"
record_NO2 = recordgroupby_df[Nitrogen_Dioxide]
record_NO2.head()

In [ ]:
# Plot the NO2 data with line chart

epochtime = earthquakes_df.iloc[earthquake, 13]
aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
earthquake_location=earthquakes_df.iloc[earthquake,2]

plt.plot(record_NO2['Date'],record_NO2['Measurement'])
plt.title('{0}'.format("NO2:" + earthquake_location + " " + aqi_event_date), fontsize =30)
plt.xlabel("Date", fontsize = 18)
plt.ylabel("NO2 (parts per billion)", fontsize =18)
plt.rcParams["figure.figsize"] = [16,8]
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=16
plt.xticks(rotation=90)
plt.axvline(x=aqi_event_date, color='r', linewidth=4)
plt.grid()
plt.show()


In [ ]:
# Extract SO2 data from the groupby dataframe

Sulfur_Dioxide = recordgroupby_df['Parameter']=="Sulfur dioxide"
record_SO2 = recordgroupby_df[Sulfur_Dioxide]
record_SO2.head()

In [ ]:
# Chart the SO2 Data

epochtime = earthquakes_df.iloc[earthquake, 13]
aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
earthquake_location=earthquakes_df.iloc[earthquake,2]

plt.plot(record_SO2['Date'],record_SO2['Measurement'])
plt.title('{0}'.format("SO2:" + earthquake_location + " " + aqi_event_date), fontsize =30)
plt.xlabel("Date", fontsize = 18)
plt.ylabel("SO2 (parts per billion)", fontsize =18)
plt.rcParams["figure.figsize"] = [16,8]
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=16
plt.xticks(rotation=90)
plt.axvline(x=aqi_event_date, color='r', linewidth=4)
plt.grid()
plt.show()

In [ ]:
# Extract Ozone from the groupby dataframe

Ozone = recordgroupby_df['Parameter']=="Ozone"
record_Ozone = recordgroupby_df[Ozone]
record_Ozone.head()

In [ ]:
#Chart the Ozone Data

epochtime = earthquakes_df.iloc[earthquake, 13]
aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
earthquake_location=earthquakes_df.iloc[earthquake,2]

plt.plot(record_Ozone['Date'],record_Ozone['Measurement'])
plt.title('{0}'.format("Ozone:" + earthquake_location + " " + aqi_event_date), fontsize =30)
plt.xlabel("Date", fontsize = 18)
plt.ylabel("Ozone (parts per million)", fontsize =18)
plt.rcParams["figure.figsize"] = [16,8]
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=16
plt.grid()
plt.axvline(x=aqi_event_date, color='r', linewidth=4)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Extract CO from the groupby dataframe

Carbon_Monoxide = recordgroupby_df['Parameter']=="Carbon monoxide"
record_CO = recordgroupby_df[Carbon_Monoxide]
record_CO.head()

In [ ]:
#Chart the Carbon Monoxide Data

epochtime = earthquakes_df.iloc[earthquake, 13]
aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
earthquake_location=earthquakes_df.iloc[earthquake,2]

plt.plot(record_CO['Date'],record_CO['Measurement'])
plt.title('{0}'.format("CO:" + earthquake_location + " " + aqi_event_date), fontsize =30)
plt.xlabel("Date", fontsize = 18)
plt.ylabel("Carbon Monoxide (parts per million)", fontsize =18)
plt.rcParams["figure.figsize"] = [16,8]
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=16
plt.xticks(rotation=90)
plt.axvline(x=aqi_event_date, color='r', linewidth=4)
plt.grid()
plt.show()